<center><h1>Covid 19 - Indian States at a Glance</h1></center>

<p>Hover over state to update charts.</p>

In [1]:
import pandas as pd
import pandas as pd
import bqplot as bqp
import numpy as np
from bqplot.market_map import MarketMap
from ipywidgets import Label, HBox, VBox, Layout
from bqplot.traits import convert_to_date

In [2]:
# Read data from covid19india website
def load_data():
    url = 'https://api.covid19india.org/csv/latest/state_wise_daily.csv'
    data = pd.read_csv(url)
    return data

In [3]:
data = load_data()
# data.head()

In [4]:
state_codes = ['AN','AP','AR','AS','BR','CH','CT','DN','DD','DL','GA','GJ','HR','HP','JK','JH',
          'KA','KL','LA','LD','MP','MH', 'MN','ML','MZ','NL','OR','PY','PB','RJ','SK','TN',
          'TG','TR','UP','UT','WB']

state_names = ['Andaman & Nicobar Is', 'Andhra Pradesh', 'Arunachal Pradesh',
          'Assam', 'Bihar', 'Chandigarh', 'Chattisgarh', 'Dadra & Nagar Haveli',
          'Daman & Diu', 'Delhi', 'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh',
          'Jammu & Kashmir', 'Jharkhand', 'Karnataka', 'Kerala', 'Ladakh', 'Lakshadweep',
          'Madhya Pradesh', 'Maharashtra', 'Manipur', 'Meghalaya', 'Mizoram', 'Nagaland',
          'Odisha', 'Puducherry', 'Punjab', 'Rajasthan', 'Sikkim', 'Tamilnadu', 'Telengana',
          'Tripura', 'Uttar Pradesh', 'Uttarakhand', 'West Bengal']

state_pop = [380581, 49577103, 1383727,
        31205576, 104099452, 1055450, 25545198, 533688,
        52076, 16787941, 1458545, 60439692, 25351462, 6864602,
        12267032, 32988134, 61095297, 33406061, 274000, 64473,
        72626809, 112374333, 2570390, 2966889, 1097206, 1978502,
        41974219, 1247953, 27743338, 68548437, 610577, 72147030, 35003674,
        3673917, 199812341, 10086292, 91276115]


confirmed_df = data[data["Status"]=="Confirmed"]
recovered_df = data[data["Status"]=="Recovered"]
deceased_df = data[data["Status"]=="Deceased"]

sdf = pd.DataFrame(columns=['state', 'ST_CD','population', 'total_cases','total_deaths', 'cpm','dpm', 'gr'])



for state_code in state_codes:
    idx = state_codes.index(state_code)
    state = state_names[idx]
    population = state_pop[idx];
    total_deaths = deceased_df[state_codes[idx]].cumsum().max()
    total_cases = confirmed_df[state_codes[idx]].cumsum().max()
    dpm = round(total_deaths * 1e6 / population,2)
    cpm = round(total_cases * 1e6 / population,2)

    d = deceased_df[state_codes[idx]].cumsum()
    ldpm = d * 1e6 / population
    delta = ldpm.iloc[-15::].diff()
    gr = delta[1::]*100/ldpm[-14::]
    gr = round(gr.mean(),2)
    prev = round(dpm*0.0666,2)
#     prev = dpm/8.472

    sdf = sdf.append({'population': population, 'total_cases': total_cases, 'total_deaths': total_deaths, 'cpm': cpm,
                      'dpm': dpm, 'state': state, 'ST_CD': state_code, 'gr': gr, 'prev': prev }, ignore_index=True)

sdf = sdf.dropna()

In [5]:
# Creating the figure to be displayed as the tooltip

sc_x = bqp.DateScale()
sc_y = bqp.LinearScale()

ax_x = bqp.Axis(scale=sc_x, grid_lines='dashed', label='Date', num_ticks = 4, label_location='end',tick_format='%b %y')
ax_y = bqp.Axis(scale=sc_y, orientation='vertical', grid_lines='dashed',
            label='Confirmed Cases', label_location='end', tick_format='.2s')

line = bqp.Lines(x=[], y=[], scales={'x': sc_x, 'y': sc_y}, colors=['orange'])
fig_tooltip = bqp.Figure(marks=[line], axes=[ax_x, ax_y], layout=Layout(width='600px', height='400px'), animation_duration=1000)

In [6]:
col = bqp.ColorScale(scheme='Reds')
ax_c = bqp.ColorAxis(scale=col, label='Cases per million')
p = bqp.market_map.MarketMap(names=sdf['ST_CD'],       
           ref_data=sdf, 
           scales={'color': col}, axes=[ax_c],
           colors = ['#ccc'], tooltip_fields=['state','cpm'],
           freeze_tooltip_location=False,
           title = 'Covid-19 - Indian States - Cases per million',
           color = sdf['cpm'],
           layout=Layout(width='600px', height='400px'))

In [7]:
# Update the tooltip chart
hovered_symbol = ''
def hover_handler(self, content):

    global hovered_symbol
    symbol = content.get('data', '')
    if(symbol != hovered_symbol):
        hovered_symbol = symbol
        if(confirmed_df.get(hovered_symbol) is not None):
            
            y1 = confirmed_df[hovered_symbol].values
            for i in range(len(y1)):
                if y1[i]<0:
                    idx = np.hstack([np.arange(i-3,i),np.arange(i+1,1+4)])
                    y1[i]=np.mean(y1[idx])

            y2 = deceased_df[hovered_symbol].values
            for i in range(len(y2)):
                if y2[i]<0:
                    idx = np.hstack([np.arange(i-3,i),np.arange(i+1,1+4)])
                    y2[i]=np.mean(y2[idx])  
            
            line.x = convert_to_date(confirmed_df['Date'].values)
            line.y = y1
            fig_tooltip.title = content.get('ref_data', {}).get('state', '')
            line2.x = convert_to_date(deceased_df['Date'].values)
            line2.y = y2
            fig_tooltip2.title = content.get('ref_data', {}).get('state', '')

p.on_hover(hover_handler)

In [8]:
# Creating the figure to be displayed as the tooltip

sc_x2 = bqp.DateScale()
sc_y2 = bqp.LinearScale()

ax_x2 = bqp.Axis(scale=sc_x2, grid_lines='dashed', label='Date', num_ticks = 4, label_location='end',tick_format='%b %y')
ax_y2 = bqp.Axis(scale=sc_y2, orientation='vertical', grid_lines='dashed',
            label='Deaths', label_location='end', tick_format='.2s')

line2 = bqp.Lines(x=[], y=[], scales={'x': sc_x2, 'y': sc_y2}, colors=['orange'])
fig_tooltip2 = bqp.Figure(marks=[line2], axes=[ax_x2, ax_y2], layout=Layout(width='600px', height='400px'), animation_duration=1000)

In [9]:
col = bqp.ColorScale(scheme='Reds')
ax_c2 = bqp.ColorAxis(scale=col, label='Deaths per million')
p2 = bqp.market_map.MarketMap(names=sdf['ST_CD'],       
           ref_data=sdf, 
           scales={'color': col}, axes=[ax_c2],
           colors = ['#ccc'], tooltip_fields=['state','dpm'],
           freeze_tooltip_location=False,
           title = 'Covid-19 - Indian States - Deaths per million',
           color = sdf['dpm'],
           layout=Layout(width='600px', height='400px'))

In [10]:
# Update the tooltip chart
# hovered_symbol = ''
# def hover_handler(self, content):

#     global hovered_symbol
#     symbol = content.get('data', '')
#     if(symbol != hovered_symbol):
#         hovered_symbol = symbol
#         if(deceased_df.get(hovered_symbol) is not None):
#             line2.x = convert_to_date(deceased_df['Date'].values)
#             line2.y = deceased_df[hovered_symbol].values
#             fig_tooltip2.title = content.get('ref_data', {}).get('state', '')
#             line.x = convert_to_date(confirmed_df['Date'].values)
#             line.y = confirmed_df[hovered_symbol].values
#             fig_tooltip.title = content.get('ref_data', {}).get('state', '')

p2.on_hover(hover_handler)

In [11]:
row1 = HBox([p,fig_tooltip])
row2 = HBox([p2,fig_tooltip2])
VBox([row1,row2])

In [12]:
cfr = 100*sdf['dpm']/sdf['cpm']

In [13]:
def_tt = bqp.Tooltip(fields=['x', 'y'], formats = ['','.2f%'], names=['State', 'CFR'])
sc_x2 = bqp.OrdinalScale()
sc_y2 = bqp.LinearScale()

ax_x2 = bqp.Axis(scale=sc_x2, grid_lines='dashed', label='State')
ax_y2 = bqp.Axis(scale=sc_y2, orientation='vertical', grid_lines='dashed', label='CFR (%)', tick_format='.2f')

scatt = bqp.Scatter(x=sdf['ST_CD'], y=cfr, scales={'x': sc_x2, 'y': sc_y2}, colors=['orange'])
cfr_fig = bqp.Figure(marks=[scatt], axes=[ax_x2, ax_y2], layout=Layout(width='1200px', height='600px'),
                     title='Case Fatality Rate', tooltip=def_tt, animation_duration=1000)
cfr_fig

Figure(animation_duration=1000, axes=[Axis(grid_lines='dashed', label='State', scale=OrdinalScale()), Axis(gri…